In [1]:
import csv
import json
import pandas as pd
from pydantic import BaseModel, ValidationError, Field, validator
from typing import Optional, Any

/Users/eskild/Library/Caches/pypoetry/virtualenvs/ski-data-qRdYxKbe-py3.10/lib/python3.10/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
route_dicts = pd.read_csv('https://docs.google.com/spreadsheets/d/1gdolhvxIzJtMCXgVVTHAZGCGcRcxhRmUm8eDZ2GHmuw/gviz/tq?tqx=out:csv&sheet=Turer', keep_default_na=False).to_dict(orient='records')
linkup_dicts = pd.read_csv('https://docs.google.com/spreadsheets/d/1gdolhvxIzJtMCXgVVTHAZGCGcRcxhRmUm8eDZ2GHmuw/gviz/tq?tqx=out:csv&sheet=Linkups', keep_default_na=False).to_dict(orient='records')
mountain_dicts = pd.read_csv('https://docs.google.com/spreadsheets/d/1gdolhvxIzJtMCXgVVTHAZGCGcRcxhRmUm8eDZ2GHmuw/gviz/tq?tqx=out:csv&sheet=Fjell', keep_default_na=False).to_dict(orient='records')

In [3]:
linkup_dicts

[{'route_id': 100000,
  'Navn': 'Austefjorden på langs',
  'geoJSON': '{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[6.719985,62.211374],[6.722474,62.208873],[6.725693,62.207072],[6.727924,62.205731],[6.729212,62.204991],[6.729341,62.204251],[6.730628,62.20385],[6.732044,62.20359],[6.734061,62.20321],[6.736979,62.202249],[6.737709,62.201609],[6.738482,62.199788],[6.737795,62.199307],[6.735048,62.199527],[6.733117,62.200148],[6.732302,62.199908]]}}',
  'Unnamed: 3': '',
  'Unnamed: 4': '',
  'Unnamed: 5': '',
  'Unnamed: 6': '',
  'Unnamed: 7': '',
  'Unnamed: 8': '',
  'Unnamed: 9': '',
  'Unnamed: 10': '',
  'Unnamed: 11': '',
  'Unnamed: 12': '',
  'Unnamed: 13': '',
  'Unnamed: 14': '',
  'Unnamed: 15': '',
  'Unnamed: 16': '',
  'Unnamed: 17': '',
  'Unnamed: 18': '',
  'Unnamed: 19': ''},
 {'route_id': 100001,
  'Navn': 'Hjørundfjorden rundt',
  'geoJSON': '{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[6.71998

In [4]:
# fields_map = {
#   'route_name': 'Rute',
#   'aspect': 'Himmelretning (topp til bunn)',
#   'area':'Område',
#   'max_start_altitude':'Høyeste startsted',
#   'avalanche_start_zones': 'Løsneområder (obligatoriske)',
#   'kast': 'KAST',
#   'dangers': 'Terrengfeller/farer',
#   'exposure': 'Eksponering',
#   'difficulty': 'Vanskelighetsgrad',
#   'comment': 'Kommentar',
#   'equipment': 'Utstyr',
#   'max_steepness':'Max bratthet',
#   'is_descent': 'Primært nedkjøring?',
#   'start_location': 'Startsted',
#   'altitude': 'Toppunkt',
#   'mountain_id': 'fjell_id'
# }

In [5]:
def string_to_bool_or_none(val:str):
    if not isinstance(val, str):
      return val

    if val == '':
      return None

    lowered_val = val.lower()

    if lowered_val == "false" or lowered_val == "usann":
        return False
    
    if lowered_val == "true" or lowered_val == "sann":
        return True

    return val

In [6]:
class Mountain(BaseModel):
  id: int = Field(..., alias="fjell_id")
  mountain_name: str =  Field(..., alias='Navn')
  altitude: int =  Field(None, alias='Høyde')
  area: str =  Field(None, alias='Område')
  
  @validator("*", pre=True)
  def parse_string(cls, val:str):
    return string_to_bool_or_none(val)


class Route(BaseModel):
  id: int = Field(..., alias="route_id")
  route_name: str =  Field(..., alias='Rute')
  max_start_altitude: int = Field(None, alias='Høyeste startsted')
  avalanche_start_zones: list = Field(None, alias='Løsneområder (obligatoriske)')
  kast: int = Field(None, alias="KAST")
  dangers: str = Field(None, alias="Terrengfeller/farer")
  exposure: str = Field(None, alias="Eksponering")
  difficulty: str = Field(None, alias="Vanskelighetsgrad")
  comment: str = Field(None, alias="Kommentar")
  equipment: str = Field(None, alias="Utstyr")
  max_steepness: Any = Field(None, alias="Max bratthet")
  is_descent: bool = Field(False, alias="Primært nedkjøring?")
  parking_location: str = Field(None, alias="Startsted")
  altitude: int = Field(None, alias="Toppunkt")
  geoJSON: dict 
  aspect: list = Field([], alias="Himmelretning (topp til bunn)")
  is_long_route: bool = False
  mountain: Optional[Mountain]

  @validator("*", pre=True)
  def parse_string(cls, val:str):
    return string_to_bool_or_none(val)

  @validator("geoJSON", pre=True)
  def string_to_dict(cls, val):
    return json.loads(val)

  @validator("aspect", pre=True)
  def split_string(cls, val):
    return val.split(' ')

  @validator("avalanche_start_zones", pre=True)
  def split_start_zones(cls, val):
    if not string_to_bool_or_none(val):
      return []

    elevation_spans = val.replace(',', '.').split('.')
    return [tuple(span.strip().split(':')) for span in elevation_spans]
  
  @validator('is_descent', pre=True)
  def empty_string_to_bool(cls, val):
    if isinstance(val, str) and val == '':
      return False
    
    return val

class Linkup(BaseModel):
  id: int = Field(..., alias="route_id")
  route_name: str =  Field(..., alias='Navn')
  geoJSON: dict 

  @validator("geoJSON", pre=True)
  def string_to_dict(cls, val):
    return json.loads(val)



In [7]:
mountains = []
invalid_mountains = []

for mountain in mountain_dicts:
  try:
    m = Mountain(**mountain)
    mountains.append(m)
  except ValidationError as e:
    error_text = (
      f'{mountain }: \n'
      f'{e} \n'
      f'-------------'
    )
    print(error_text)

    invalid_mountains.append(mountain)

print(f'Invalid mountains: {len(invalid_mountains)}')
print(f'Valid mountain: {len(mountains)}')

Invalid mountains: 0
Valid mountain: 53


In [8]:
routes = []
invalid_routes = []

for route in route_dicts:
  try:
    mountain = next((mountain for mountain in mountains if mountain.id == route['fjell_id']), None)
    route['mountain'] = mountain
    r = Route(**route)
    routes.append(r)
  except ValidationError as e:
    error_text = (
      f'{route }: \n'
      f'{e} \n'
      f'-------------'
    )
    print(error_text)

    invalid_routes.append(route)

{'route_id': 74, 'fjell_id': 13, 'Fjell': 'Grøtdalstind', 'Rute': 'Normalveien', 'Toppunkt': '1330', 'Område': 'Kolåshalvøya', 'Primært nedkjøring?': 'FALSE', 'Himmelretning (topp til bunn)': 'V S', 'Startsted': '', 'Høyeste startsted': '250', 'Løsneområder (obligatoriske)': '930-970:30-35. 1170-1250:30-45', 'Max bratthet': '', 'Terrengfeller/farer': '', 'KAST': '', 'Eksponering': '', 'Vanskelighetsgrad': '', 'Utstyr': '', 'Kommentar': '', 'Hvem': '', 'geoJSON': '', 'Unnamed: 20': '', 'Unnamed: 21': '', 'Unnamed: 22': '', 'Unnamed: 23': '', 'Unnamed: 24': '', 'Unnamed: 25': '', 'Unnamed: 26': '', 'Unnamed: 27': '', 'Unnamed: 28': '', 'Unnamed: 29': '', 'Unnamed: 30': '', 'Unnamed: 31': '', 'Unnamed: 32': '', 'Unnamed: 33': '', 'Unnamed: 34': '', 'Unnamed: 35': '', 'Unnamed: 36': '', 'mountain': Mountain(id=13, mountain_name='Grøtdalstind', altitude=1330, area='Kolåshalvøya')}: 
1 validation error for Route
geoJSON
  Expecting value: line 1 column 1 (char 0) (type=value_error.jsondecode

In [9]:
print(f'Invalid routes: {len(invalid_routes)}')
print(f'Valid routes: {len(routes)}')

Invalid routes: 6
Valid routes: 77


In [10]:
linkups = []
invalid_linkups = []

for linkup in linkup_dicts:
  try:
    r = Linkup(**linkup)
    linkups.append(r)
  except ValidationError as e:
    error_text = (
      f'{linkup }: \n'
      f'{e} \n'
      f'-------------'
    )
    print(error_text)

    invalid_linkups.append(linkup)

print(f'Invalid long routes: {len(invalid_linkups)}')
print(f'Valid long routes: {len(linkups)}')

Invalid long routes: 0
Valid long routes: 3


In [11]:
route_dicts = [route.dict() for route in routes]
with open('routes.json', 'w') as file:
    file.write(json.dumps(route_dicts))

linkup_dicts = [route.dict() for route in linkups]
with open('linkups.json', 'w') as file:
    file.write(json.dumps(linkup_dicts))
